In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Resources: 
* https://www.analyticsvidhya.com/blog/2021/09/adaboost-algorithm-a-complete-guide-for-beginners/
* sklearn documentation
* Multi class adaboost Ji Zhu
* AdaBoost-CNN Aboozar

### Imports

In [2]:
import matplotlib.pyplot as plt

import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers.experimental import RMSprop, Adam
import gc
import random
import math
import keras.backend as K
from sklearn.model_selection import train_test_split

random.seed(42)


#import libraries
import tensorflow as tf
import keras
from keras.models import Sequential , Model
from keras.applications.vgg16 import VGG16
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Input, Add, Activation, ZeroPadding2D, BatchNormalization, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D , Dense, Conv2D, MaxPool2D , Flatten , concatenate ,DepthwiseConv2D, GlobalAveragePooling2D , ReLU

### Read Data

In [3]:
def read_data(list, photo_size):  
    data = []
    for i in range(len(list)):
        for filename in os.listdir(list[i]):
            img  = cv2.imread(list[i] + filename)
            img = cv2.resize(img,(photo_size,photo_size))
            data.append([img, [i%2, abs(i%2 - 1)] ])
        
    
    random.shuffle(data)
    x = []
    y = []
    for i in range(len(data)):
        x.append(data[i][0])
        y.append(data[i][1])
    
    x = np.array(x)
    y = np.array(y)
    x = x / 255
    return x, y

### Xception

In [4]:
def make_model():
    xception = tf.keras.applications.Xception(input_shape=(224, 224,3), include_top=False, weights="imagenet" )
    
    model = tf.keras.models.Sequential([
    xception,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, "relu"),
    tf.keras.layers.Dense(64, "relu"),
    tf.keras.layers.Dense(2, "softmax")
    ])
      
    return model


### Data split

In [5]:
ta = "/kaggle/input/autistic-children-facial-data-set/train/autistic/"
tn = "/kaggle/input/autistic-children-facial-data-set/train/non_autistic/"
testa = "/kaggle/input/autistic-children-facial-data-set/test/autistic/"
testn = "/kaggle/input/autistic-children-facial-data-set/test/non_autistic/"
va = "/kaggle/input/autistic-children-facial-data-set/valid/autistic/"
vn = "/kaggle/input/autistic-children-facial-data-set/valid/non_autistic/"

lis = [tn, ta , testn, testa, vn, va]


x, y = read_data(lis , 224)
print(x.shape)
print(y.shape)

del ta , tn , testa, testn, va, vn, lis

gc.collect()

(2936, 224, 224, 3)
(2936, 2)


52

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.85, test_size = 0.15, stratify = y)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, train_size = 0.9, test_size = 0.1, stratify = y_train)
del x, y
gc.collect()

23

In [7]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_valid.shape)
print(y_valid.shape)

(2245, 224, 224, 3)
(2245, 2)
(441, 224, 224, 3)
(441, 2)
(250, 224, 224, 3)
(250, 2)


### MoibleNet

In [8]:
def depth_block(x, strides):
    x = DepthwiseConv2D(3,strides=strides,padding='same',  use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
def single_conv_block(x,filters):
    x = Conv2D(filters, 1,use_bias=False)(x)
    x= BatchNormalization()(x)
    x = ReLU()(x)
    return x
def combo_layer(x,filters,strides):
    x = depth_block(x,strides)
    x = single_conv_block(x, filters)
    return x
def MobileNet(input_shape=(224,224,3),n_classes = 2):
    input =layers.Input( input_shape)
    x = Conv2D(32,3,strides=(2,2),padding = 'same', use_bias=False) (input)
    x =  BatchNormalization()(x)
    x = ReLU()(x)
    x = combo_layer(x,64, strides=(1,1))
    x = combo_layer(x,128,strides=(2,2))
    x = combo_layer(x,128,strides=(1,1))
    x = combo_layer(x,256,strides=(2,2))
    x = combo_layer(x,256,strides=(1,1))
    x = combo_layer(x,512,strides=(2,2))
    for _ in range(5):
        x = combo_layer(x,512,strides=(1,1))
    x = combo_layer(x,1024,strides=(2,2))
    x = combo_layer(x,1024,strides=(1,1))
    x = GlobalAveragePooling2D()(x)
    output = Dense(n_classes,activation='softmax')(x)
    model = Model(input, output)
    return model

# SAMME Algorithm

In [267]:
def samme_evalution(x,y,n_e, estimators_alphas, estimators):
    predicted = []
    for i in range(n_e):
        predicted.append(estimators[i].predict(x))
        gc.collect()
    _m5ra = []   
    for i in range(n_e):
        _m5ra.append([estimators_alphas[i] * j for j in predicted[i].round()])
    _m5ra = np.array(_m5ra).sum(axis = 0)

    acc = 0
    for i in range(len(x)):
       if (np.argmax(_m5ra[i])) == (np.argmax(y[i])):
            acc += 1 
    return acc / y.shape[0]

In [269]:
estimators = []  # list of models
estimators_weights = [] # list of CNN weights of each model
estimators_alphas = [] # 
sample_weights = [] # list of sample weights of each model
n_e = 6 # number of models
gc.collect()

for i in range(0, n_e):
    gc.collect()
    # models after base
    if i > 0 :
        estimator = make_model()
        gc.collect()
        estimator.set_weights(estimators_weights[i-1])
        gc.collect()
        estimator.compile(optimizer="RMSprop",loss=tf.keras.losses.BinaryCrossentropy(),metrics=['acc'])
        gc.collect()
        sample_weight = sample_weights[i-1]
        gc.collect()
    # base model
    else:
        estimator = make_model()
        gc.collect()
        estimator.compile(optimizer="RMSprop",loss=tf.keras.losses.BinaryCrossentropy(),metrics=['acc'])
        gc.collect()
        sample_weight = np.ones( x_train.shape[0] ) / x_train.shape[0]
        gc.collect()
    
        
    print(f"Estimator no. {i+1} \n\n")
    gc.collect()
    gc.collect()
    
    # base model and others could have different epochs
    if i > 0:
        estimator.fit(x_train, y_train, sample_weight = sample_weight, validation_data = [x_valid , y_valid], epochs = 1, batch_size = 32)
    else:
        estimator.fit(x_train, y_train, sample_weight = sample_weight, validation_data = [x_valid , y_valid], epochs = 1, batch_size = 32)
    gc.collect()
    gc.collect()
    
    # simple evaluation for each model
    print("\n\n")
    print(f"Train Acc: {estimator.evaluate(x_train, y_train)}\n\n")
    gc.collect()
    gc.collect()
    print(f"Test Acc: {estimator.evaluate(x_test, y_test)}\n\n")
    gc.collect()
    gc.collect()
    print(f"Valid Acc: {estimator.evaluate(x_valid, y_valid)}\n\n")
    gc.collect()
    gc.collect()
    
    
    # Step 2: Calculate the err equation
    #print("start step 2")
    predicted = (estimator.predict(x_train)).round()
    gc.collect()
    gc.collect()
    
    # 1 if the model classifiy it wrong, 0 then it is calssified it right
    incorrect = np.array([1 if i.all() == True else 0 for i in predicted != y_train ])
    del predicted
    gc.collect()
    
    error = np.dot(incorrect, sample_weight) / np.sum(sample_weight)
    #print(f"err = {error}\n\n")
    gc.collect()

    ############ we need to change this condition  ###############
    if error >= 1 - 1 / 2 and i > 0:
        print("stoped boosting")
        break
    
    #print("start step 3")
    # step 3: caluclate the alpha
    lr = K.eval(estimator.optimizer.lr)
    alpha = lr*(math.log((1-error)/error)) + math.log(2-1)
    del error, lr
    gc.collect()
    
    #print("start step 4")
    # step 4: update the weights
    sample_weight *= np.exp(alpha * incorrect) 
    gc.collect()
    sample_weight /= sample_weight.sum()
    
    # appending and cleaning before repeating
    estimators_alphas.append(alpha)
    del alpha , incorrect
    gc.collect()
    
    estimators.append(estimator)
    gc.collect()
    
    estimators_weights.append(estimator.get_weights())
    del estimator
    gc.collect()
    
    sample_weights.append(sample_weight)
    del sample_weight
    gc.collect()
    
    #print("finish \n")

del estimators_weights
gc.collect()

Estimator no. 1 


71/71 [==============================] - 36s 397ms/step - loss: 2.4479e-04 - acc: 0.7461 - val_loss: 1.4362 - val_acc: 0.7360



71/71 [==============================] - 6s 79ms/step - loss: 1.0727 - acc: 0.7915
Train Acc: [1.072749137878418, 0.7915367484092712]


14/14 [==============================] - 1s 78ms/step - loss: 1.5073 - acc: 0.7188
Test Acc: [1.507330298423767, 0.7188208699226379]


8/8 [==============================] - 1s 77ms/step - loss: 1.4362 - acc: 0.7360
Valid Acc: [1.4361693859100342, 0.7360000014305115]


71/71 [==============================] - 6s 75ms/step
Estimator no. 2 


71/71 [==============================] - 37s 401ms/step - loss: 1.5731e-04 - acc: 0.8508 - val_loss: 0.7050 - val_acc: 0.8440



71/71 [==============================] - 6s 78ms/step - loss: 0.3223 - acc: 0.9158
Train Acc: [0.32231205701828003, 0.9158129096031189]


14/14 [==============================] - 1s 77ms/step - loss: 0.9051 - acc: 0.8050
Test Acc: [0.9051039218

0

In [270]:
print(f" Train Acc: {samme_evalution(x_train,y_train,n_e, estimators_alphas, estimators)}")
print(f" Test Acc: {samme_evalution(x_test,y_test,n_e, estimators_alphas, estimators)}")
print(f" Validation Acc: {samme_evalution(x_valid,y_valid,n_e, estimators_alphas, estimators)}")

71/71 [==============================] - 5s 77ms/step
 Train Acc: 0.9848552338530067
14/14 [==============================] - 1s 75ms/step
 Test Acc: 0.8253968253968254
8/8 [==============================] - 1s 74ms/step
 Validation Acc: 0.86


# Notes


# SAMME.R

In [15]:
# Step 3: Calculate h
def Hk(predicted, k = 2):
    predicted = np.array(predicted)
    predicted[predicted < np.finfo(predicted.dtype).eps] = np.finfo(predicted.dtype).eps
    log_predict = np.log(predicted)
    h = (k-1) * (log_predict - 1/k * log_predict.sum())
    
    return h

def evaluate(x, y, estimators, n_e, hk = None, k = 2):
    if hk == None:
        hk = []
        for i in range(0,n_e):
            hk.append(estimators[i].predict(x))
            gc.collect()

        hk = np.array(hk)
        hk[hk < np.finfo(hk.dtype).eps] = np.finfo(hk.dtype).eps
        gc.collect()
        log_prop = np.log(hk)
        gc.collect()
        intermediate = (k-1) * (log_prop - 1/k * log_prop.sum())
        gc.collect()
        intermediate = intermediate.sum(axis = 0)
    else:
        intermediate = np.array(hk).sum(axis = 0)
    acc = 0
    for i in range(len(y)):
       if (np.argmax(intermediate[i])) == (np.argmax(y[i])):
            acc += 1 
    print(f"Accuracy : {acc/y.shape[0]}")
    return acc/y.shape[0]

In [16]:
gc.collect()
gc.collect()

estimators = [] # list of models
estimators_weights = [] # list of CNN weights of each model
sample_weights = [] # list of sample weights of each model
#H = [] # list of h of each model

estimators_pred = []

y_labels = [ i[0] for i in y_train]
y_coding = np.array([1 if i == 1 else 1.01 for i in y_labels])

n_e = 4
gc.collect()

for i in range(0, n_e):
    gc.collect()
    # models after base
    if i > 0 :
        estimator = make_model() #MobileNet()
        gc.collect()
        estimator.set_weights(estimators_weights[i-1])
        gc.collect()
        estimator.compile(optimizer="RMSprop",loss=tf.keras.losses.BinaryCrossentropy(),metrics=['acc'])
        gc.collect()
        sample_weight = sample_weights[i-1]
        gc.collect()
    
    # base model
    else:
        estimator = make_model() #MobileNet()
        gc.collect()
        estimator.compile(optimizer="RMSprop",loss=tf.keras.losses.BinaryCrossentropy(),metrics=['acc'])
        gc.collect()
        sample_weight = np.ones(x_train.shape[0]) / x_train.shape[0]
        gc.collect()
    
        
    print(f"Estimator no. {i+1} \n\n")
    gc.collect()
    gc.collect()
    
    # base model and others could have different epochs
    if i > 0:
        estimator.fit(x_train, y_train, sample_weight=sample_weight, validation_data = [x_valid , y_valid], epochs = 1, batch_size = 64)
    else:
        estimator.fit(x_train, y_train, sample_weight=sample_weight, validation_data = [x_valid , y_valid], epochs = 2, batch_size = 8)
    gc.collect()
    gc.collect()
    
    # simple evaluation for each model
    print("\n\n")
    print(f"Train Acc: {estimator.evaluate(x_train, y_train)}\n\n")
    gc.collect()
    gc.collect()
    print(f"Test Acc: {estimator.evaluate(x_test, y_test)}\n\n")
    gc.collect()
    gc.collect()
    print(f"Valid Acc: {estimator.evaluate(x_valid, y_valid)}\n\n")
    gc.collect()
    gc.collect()
    
    ############# first we need to calculate the alpha for early stopping ############
    #################################################################################
    ################### DONT, DONT, DONT FOOOOOORGEEEEEEET #########################
    ################### DONT, DONT, DONT FOOOOOORGEEEEEEET #########################
    ###############################################################################
    
    # Step 2: get class predicted prop
    #print("start step 2")
    predicted = (estimator.predict(x_train))
    gc.collect()
    gc.collect()
    
    prop = np.array([predicted[i][0] for i in range(len(y_train))])
    prop[prop < np.finfo(prop.dtype).eps] = np.finfo(prop.dtype).eps
    gc.collect()
    gc.collect()
    
    # Step 3: Calculate h
    #predicted = np.array(predicted)
    #H.append(Hk(predicted))
    #del predicted
    #gc.collect()
    #gc.collect()
    
    #print("start step 4")
    # step 4: update the weights
    lr =  1 #K.eval(estimator.optimizer.lr)
    intermediate_value = np.exp(-lr * (((2 - 1) / 2) * (y_coding* np.log(prop))))
    gc.collect()
    sample_weight = sample_weight * intermediate_value
    gc.collect()
    sample_weight = sample_weight / sample_weight.sum()
    del intermediate_value, prop, lr
    gc.collect()
    
    # appending and cleaning before repeating
    estimators.append(estimator)
    gc.collect()
    
    estimators_weights.append(estimator.get_weights())
    del estimator
    gc.collect()
    
    sample_weights.append(sample_weight)
    del sample_weight
    gc.collect()
    #print("finish \n")

del estimators_weights
gc.collect()

Estimator no. 1 


Epoch 1/2
281/281 [==============================] - 44s 128ms/step - loss: 2.7808e-04 - acc: 0.6592 - val_loss: 8.0220 - val_acc: 0.6160
Epoch 2/2
281/281 [==============================] - 35s 123ms/step - loss: 2.1919e-04 - acc: 0.7813 - val_loss: 0.4460 - val_acc: 0.7960



71/71 [==============================] - 6s 78ms/step - loss: 0.4085 - acc: 0.8178
Train Acc: [0.40847888588905334, 0.8178173899650574]


14/14 [==============================] - 1s 78ms/step - loss: 0.5031 - acc: 0.7619
Test Acc: [0.5031496286392212, 0.761904776096344]


8/8 [==============================] - 1s 77ms/step - loss: 0.4460 - acc: 0.7960
Valid Acc: [0.4459714889526367, 0.7960000038146973]


71/71 [==============================] - 6s 75ms/step
Estimator no. 2 


36/36 [==============================] - 35s 765ms/step - loss: 1.9703e-05 - acc: 0.8673 - val_loss: 0.4036 - val_acc: 0.8400



71/71 [==============================] - 6s 79ms/step - loss: 0.2849 - acc: 0.8846
Train Acc

0

In [17]:
print(f" Train Acc: {evaluate(x_train, y_train, estimators, n_e, k = 2)}")
print(f" Test Acc: {evaluate(x_test, y_test, estimators, n_e, k = 2)}")
print(f" Validation Acc: {evaluate(x_valid, y_valid, estimators, n_e, k = 2)}")

71/71 [==============================] - 5s 76ms/step
Accuracy : 0.8832962138084632
 Train Acc: 0.8832962138084632
14/14 [==============================] - 1s 75ms/step
Accuracy : 0.81859410430839
 Test Acc: 0.81859410430839
8/8 [==============================] - 1s 75ms/step
Accuracy : 0.848
 Validation Acc: 0.848


In [ ]:
gc.collect()